## Nate Finck

Changes made: 

* the reshape function in the last code chunk was not working so I changed the (8,8) to (8,104). 

In [1]:
pip install chess

     |████████████████████████████████| 147 kB 5.5 MB/s 


# **Step 1| Prepare Data**
1. Download the data

In [2]:
!wget https://www.pgnmentor.com/players/Carlsen.zip
!unzip Carlsen.zip

--2021-11-20 21:10:58--  https://www.pgnmentor.com/players/Carlsen.zip
Resolving www.pgnmentor.com (www.pgnmentor.com)... 65.254.227.240
Connecting to www.pgnmentor.com (www.pgnmentor.com)|65.254.227.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 900358 (879K) [application/zip]
Saving to: ‘Carlsen.zip’

Carlsen.zip         100%[===================>] 879.26K  5.43MB/s    in 0.2s    

2021-11-20 21:10:59 (5.43 MB/s) - ‘Carlsen.zip’ saved [900358/900358]

Archive:  Carlsen.zip
  inflating: Carlsen.pgn             


In [3]:
import chess.pgn
pgn = open("Carlsen.pgn")
sides = []
games = []
length = 100
for i in range(length):
    try:
        if chess.pgn.read_game(pgn).mainline_moves():
            games.append(chess.pgn.read_game(pgn).mainline_moves())
            sides.append(chess.pgn.read_game(pgn).headers["White"])
    except:
        print(i,chess.pgn.read_game(pgn))
        pass
len(games)

100

In [4]:
X = []
y = []
counter2 = 0
for game in games:
    board = chess.Board()
    white = sides[counter2]
    if white == 'Carlsen,Magnus':
        remainder = 0
    else:
        remainder = 1
    counter = 0
    for move in game:
        if counter % 2 == remainder:
            X.append(board.copy())
        board.push(move)
        if counter % 2 == remainder:
            y.append(board.copy())
        counter += 1
    counter2 += 1

In [5]:
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1,0],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0,1],
}
def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo
def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows
import numpy as np
for i in range(len(X)):
    X[i] = translate(make_matrix(X[i]),chess_dict)
for i in range(len(y)):
    y[i] = translate(make_matrix(y[i]),chess_dict)
X = np.array(X)
y = np.array(y)

# **Step 2| MagnusGAN:**

In [6]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.initializers import RandomNormal
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Activation,Reshape
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dropout

In [7]:
def define_discriminator():
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape)
    in_target_image = Input(shape=image_shape)
    merged = concatenate([in_src_image, in_target_image])
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    model = Model(inputs = [in_src_image, in_target_image], outputs = patch_out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [8]:
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g
 
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.5)(g, training=True)
    g = concatenate([g, skip_in])
    g = Activation('relu')(g)
    return g

In [9]:
def define_generator(image_shape=(8,8,13)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e2)
    b = Activation('relu')(b)
    d6 = decoder_block(b, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    g = Conv2DTranspose(13, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('softmax')(g)
    model = Model(in_image, out_image)
    return model

In [10]:
def define_gan(g_model, d_model, image_shape):
    d_model.trainable = False
    in_src = Input(shape=image_shape)
    gen_out = g_model(in_src)
    dis_out = d_model([in_src, gen_out])
    model = Model(in_src, [dis_out, gen_out])
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

# **Step 3| Prepare for execution:**

In [11]:
def generate_real_samples(dataset, n_samples, patch_shape):
    trainA, trainB = dataset
    ix = randint(0, trainA.shape[0], n_samples)
    X1, X2 = trainA[ix], trainB[ix]
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y
 
def generate_fake_samples(g_model, samples, patch_shape):
    X = g_model.predict(samples)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [12]:
def train(d_model, g_model, gan_model, dataset, n_epochs=1, n_batch=1):
    n_patch = d_model.output_shape[1]
    trainA, trainB = dataset
    bat_per_epo = int(len(trainA) / n_batch)
    n_steps = bat_per_epo * n_epochs
    for i in range(n_steps):
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    if (i+1) % (bat_per_epo * 10) == 0:
        clear_output()

In [13]:
image_shape = (8,8,13)
d_model = define_discriminator()
g_model = define_generator()
gan_model = define_gan(g_model, d_model, image_shape)
train(d_model, g_model, gan_model, [X,y])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, d1[0.347] d2[0.347] g[14.919]
>2, d1[0.346] d2[0.347] g[14.865]
>3, d1[0.346] d2[0.347] g[14.806]
>4, d1[0.346] d2[0.347] g[14.754]
>5, d1[0.346] d2[0.347] g[14.760]
>6, d1[0.346] d2[0.347] g[14.702]
>7, d1[0.346] d2[0.347] g[14.729]
>8, d1[0.346] d2[0.347] g[14.604]
>9, d1[0.346] d2[0.347] g[14.477]
>10, d1[0.346] d2[0.347] g[14.401]
>11, d1[0.346] d2[0.347] g[14.452]
>12, d1[0.346] d2[0.347] g[14.297]
>13, d1[0.346] d2[0.347] g[14.242]
>14, d1[0.346] d2[0.347] g[14.063]
>15, d1[0.346] d2[0.347] g[14.200]
>16, d1[0.346] d2[0.347] g[14.002]
>17, d1[0.346] d2[0.347] g[14.071]
>18, d1[0.346] d2[0.347] g[13.786]
>19, d1[0.346] d2[0.347] g[13.849]
>20, d1[0.346] d2[0.347] g[13.744]
>21, d1[0.346] d2[0.347] g[13.235]
>22, d1[0.346] d2[0.347] g[13.484]
>23, d1[0.346] d2[0.347] g[13.326]
>24, d1[0.346] d2[0.347] g[13.349]
>25, d1[0.347] d2[0.347] g[13.160]
>26, d1[0.347] d2[0.347] g[13.645]
>27, d1[0.347] d2[0.347] g[12.827]
>28, d1[0.347] d2[0.347] g[12.775]
>29, d1[0.347] d2[0.347] g[12

# **Step 4| Observe results:**

In [20]:
import random
flatten = lambda l: [item for sublist in l for item in sublist]
instance = random.randint(1,len(X)-1)
state = X[instance].reshape(1,8,8,13)
action = gan_model.predict(state)[1]
def retranslate(action):
    board = []
    flatten_action = flatten(flatten(action))
    for i in range(len(flatten_action)):
        new_set = np.zeros((13,))
        max_index = list(flatten_action[i]).index(max(flatten_action[i]))
        new_set[max_index] = 1
        board.append(new_set)
    for i in range(len(board)):
        # print(board[i])
        board[i] = tuple(board[i])
    print(np.array(board).shape)
    board = np.array(board).reshape(8,104)
    print(board)
        
retranslate(action)

(64, 13)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0.